In [3]:
import pandas as pd
import numpy as np

In [4]:
DATA_PATH = './'

In [5]:
iris_df = pd.read_csv(DATA_PATH + 'data/iris_encoded.csv', index_col=0)

In [6]:
list_weight = np.loadtxt(DATA_PATH + 'data/weights.csv', delimiter = ',')

In [7]:
iris_data = iris_df.iloc[:-1, :]
iris_labels = iris_df.iloc[-1, :].astype(int)

In [8]:
iris_data.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.393,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.865,0.0,0.393,NaN,NaN,0.393,NaN,NaN,0.865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.865,NaN,NaN,0.393,NaN,NaN,0.393,NaN,0.865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,0.393,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.393,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [9]:
def model_data(lat_ne):
    
    train_stack = np.where(lat_ne >= 0, lat_ne, -1)
    
    return train_stack      # (150, 40)

In [10]:
def leaky_integrate_and_fire(mem, x, w, beta, threshold=1):
    spk = (mem > threshold) # if membrane exceeds threshold, spk=1, else, 0
    # mem = beta * mem + w*x - spk*threshold
    if spk:
        mem = 0
    else:
        mem = beta * mem + w*x
    # mem = beta * mem + w*x
    return spk, mem

In [291]:
def LIF_SNN(n, data, weight, v_spike = 0.25):
    l = len(data)

    V_min = 0
    r = 5
    tau = 2.5
    dt = 0.01
    beta = 1 - dt / tau
    t_max = 10
    time_stamps = t_max / dt
    # time_relax = 10
    v = np.zeros((n, l, int(time_stamps)))
    t_post = np.zeros((n, l))
    v[:, :, 0] = V_min
    
    # data_spike_list = np.zeros((l, n, len(data[0]), int(time_stamps)))
    data_spike_list_sum = np.zeros((l, n, int(time_stamps)))

    for u in range(l):          # data point
        for ni in range(n):     # 3 classes
            f0 = (np.round(data[u][np.newaxis].T, 3) * 1000).astype(int)      # (40, 1)
            f1 = np.tile(np.arange(1000), (40, 1))                            # (40, 1)
            f2 = np.where(((f1 == f0) & (f0 >= 0)), 1, 0)                      # (40, 1000)        1 wherever the spike is
            f2 = f2 * weight[ni][np.newaxis].T
            # data_spike_list[u][ni] = f2
            data_spike_list_sum[u][ni] = np.sum(f2, axis = 0)

    for u in range(l):          # data point
        for step in range(int(time_stamps) - 1):
            for ni in range(n):     # 3 classes
                spike_list = data_spike_list_sum[u][ni]
                (spk_out, mem_out) = leaky_integrate_and_fire(v[ni, u, step], r * dt/tau * spike_list[step], 1, beta, threshold=v_spike)

                if spk_out:
                    t_post[ni, u] = step

                v[ni, u, step + 1] = mem_out

    return v, t_post

In [83]:
# def network_hw(data, weight, time_stamps, v_spike, beta, w, in_n, out_n):
#     v = np.zeros(out_n)

#     for step in range(int(time_stamps) - 1):
#         for ni in range(n):     # 3 classes
#             spike_list = data_spike_list_sum[u][ni]
#             (spk_out, mem_out) = leaky_integrate_and_fire(v[ni, u, step], w * spike_list[step], 1, beta, threshold=v_spike)

#             if spk_out:
#                 t_post[ni, u] = step

#             v[ni, u, step + 1] = mem_out

In [11]:
def float_to_lat_step (data):
    data = np.round(data, 3)
    data = data * 1000
    data = data.astype(int)
    return data

In [55]:
def LIF_SNN_hw(n_classes, data, weight, v_spike = 0.25):
    dt = 0.01
    beta = 0.996
    t_max = 10
    time_stamps = int(t_max / dt)
    in_neur_n = len(data[0])
    n_inputs = len(data)

    w = 0.0234375                     # ~r*dt/tau = 2^(-6) + 2^(-7)

    data = float_to_lat_step(data)

    t_post = np.zeros((n_classes, n_inputs))

    for i in range(n_inputs):
        data_i = data[i]
        out_neur_input = np.zeros((time_stamps, n_classes))

        v = np.zeros(n_classes)

        for t in range(int(time_stamps)):
            in_neur_act = np.tile(t, (in_neur_n,)) == data_i
            
            for j in range(n_classes):
                acc = 0
                for k in range(in_neur_n):
                    if in_neur_act[k]:
                        acc += weight[j][k]
                out_neur_input[t][j] = acc

                (spk_out, mem_out) = leaky_integrate_and_fire(v[j], w * out_neur_input[t][j], 1, beta, threshold=v_spike)
                
                if spk_out:
                    t_post[j][i] = t

                v[j] = mem_out

    return None, t_post

In [56]:
lat_ne = np.transpose(iris_data.values)       # (150, 40)
test_stack = model_data(lat_ne)

In [57]:
(_,out) = LIF_SNN_hw(3, test_stack, list_weight)

In [58]:
out.shape

(3, 150)

In [59]:
accuracy_snn(out, iris_labels.values)

accur.: 95.33 %


/tmp/ipykernel_105163/3758591641.py:6: RuntimeWarning: invalid value encountered in cast
  final_test = np.full([len(spike_time[0])], np.nan).astype(int)


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [19]:
def accuracy_snn(spike_time, iris_labels):
    
    target_type = iris_labels
    
    spike_time_ = np.where(spike_time > 0, np.array(([1], [2], [3])), np.nan)
    final_test = np.full([len(spike_time[0])], np.nan).astype(int)
    for i in range(len(spike_time[0])):
        try:
            final_test[i] = spike_time_[:, i][spike_time[:, i] == np.min(spike_time[:, i][spike_time[:, i] > 0])][0]
        except:
            final_test[i] = 0
    
    ac = np.sum(np.where(final_test == target_type, 1, 0)) / len(target_type)

    print('accur.:', np.round(ac * 100, 2), '%')

    return final_test, target_type

In [86]:
lat_ne = np.transpose(iris_data.values)       # (150, 40)

In [292]:
test_stack = model_data(lat_ne)

res = LIF_SNN(3, test_stack, list_weight)
spike_time = res[1]

accuracy_snn(spike_time, iris_labels.values)

accur.: 94.67 %


/tmp/ipykernel_2698395/3758591641.py:6: RuntimeWarning: invalid value encountered in cast
  final_test = np.full([len(spike_time[0])], np.nan).astype(int)


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [88]:
test_stack[0]

array([-1.   , -1.   ,  0.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
       -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.751,
        0.393, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
       -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,  0.393,  0.751,
       -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ])